In [4]:
from PIL import Image, ImageFont, ImageDraw 
import numpy as np
import pandas as pd

import colorsys
import random

In [14]:
# Variables:
seed_for_colors = 42
original_image_path = "/Users/benne/Downloads/HIDA-ufz_image_challenge/photos_annotated/2019_0626_080253_001.jpg.jpg"
annotated_image_path = "/Users/benne/Downloads/HIDA-ufz_image_challenge/photos_annotated/2019_0626_080253_001.jpg.png"
feature_legend_path = "/Users/benne/Downloads/HIDA-ufz_image_challenge/features_legend.csv"
outcome_path = "example.png"

In [15]:
# Generate random colors:
def random_colors(N, bright=True):
    """
    Generate random colors.
    To get visually distinct colors, generate them in HSV space then
    convert to RGB.
    """
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.shuffle(colors)
    return colors

In [20]:
def visualize(original_image_path: str, annotated_image_path: str,feature_legend_path: str, outcome_path: str, seed = 42):
    # load original image and image with annotations
    original_image = Image.open(original_image_path)
    image_annotations = Image.open(annotated_image_path).convert('RGB')
    
    # set annotated image to semi-transparent
    image_annotations.putalpha(alpha = 128)
    
    # create nice looking colors
    random.seed(seed)
    colors = random_colors(20)
    
    # transform image so one can work with it
    image_transformed = np.array(image_annotations)

    height = np.array(image_annotations).shape[0]
    width = np.array(image_annotations).shape[1]

    image_transformed = image_transformed.reshape(height*width, 4)

    # Make black pixels transparent:
    image_transformed = [[0, 0, 0, 0]  if v == [0, 0, 0, 128] else v for v in image_transformed.tolist()]

    # Resetting RGB ID's to nice colors:
    for i in range(1,21) :
        r = colors
        image_transformed = [[colors[i][0]*255, colors[i][1]*255, colors[i][2]*255, 128]  if v == [i, i, i, 128] else v for v in image_transformed]


    #Reshape and make it Image type again
    image_transformed = np.array(image_transformed, dtype = 'uint8')
    image_transformed = image_transformed.reshape(height, width, 4)
    
    #Put original image in the background and paste annotations over it

    img = Image.fromarray(image_transformed)
    background = original_image
    background.putalpha(alpha = 255)
    background.paste(img, (0, 0), img)
    
    output_image_without_writing = background
    
    # Calculate percentages
    rgb_ids = pd.read_csv(feature_legend_path)
    
    image_annotations = Image.open(annotated_image_path).convert('RGB')
    image_annotations = np.array(image_annotations)
    height = np.array(image_annotations).shape[0]
    width = np.array(image_annotations).shape[1]

    image_for_counting = image_annotations.reshape(height*width, 3).tolist()
    
    rgb_ids['percentage'] = 0
    for i in range(1, 21):
        rgb_ids['percentage'][rgb_ids['id_rgb'] == i] = (image_for_counting.count([i, i, i])/(height*width))*100
    
    # Write percentages on image
    dataframe_for_writing = rgb_ids[rgb_ids['percentage'] > 0]
    
    #Select Arial as font
    title_font = ImageFont.truetype(r'C:\Users\System-Pc\Desktop\arial.ttf', 20)

    for i in range(0, dataframe_for_writing.shape[0]):
        title_text = f"{dataframe_for_writing[' label'].iloc[i]}: {round(dataframe_for_writing['percentage'].iloc[i], 2)}%"

        image_editable = ImageDraw.Draw(output_image_without_writing)
        image_editable.text((15,15 + i*25), title_text, (0, 0, 0), font=title_font)
        
    output_image = output_image_without_writing
    
    output_image.save(outcome_path, "PNG")

In [21]:
visualize(original_image_path, annotated_image_path, feature_legend_path, outcome_path, seed_for_colors)

C:\Users\benne\anaconda3\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\benne\anaconda3\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
